In [2]:
import torch, botorch, gpytorch
print("Torch:", torch.__version__)
print("BoTorch:", botorch.__version__)
print("GPyTorch:", gpytorch.__version__)
from botorch.acquisition.logei import qLogNoisyExpectedImprovement
print("qLogNoisyExpectedImprovement import OK")


C:\Users\jonghyun\anaconda3\envs\boenv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Torch: 2.6.0+cu124
BoTorch: 0.15.1
GPyTorch: 1.14
qLogNoisyExpectedImprovement import OK


In [8]:
pip install pandas

   ---------------------------------------- 0.0/11.3 MB ? eta -:--:--
   -------- ------------------------------- 2.4/11.3 MB 12.2 MB/s eta 0:00:01
   ---------------- ----------------------- 4.7/11.3 MB 11.4 MB/s eta 0:00:01
   ------------------------- -------------- 7.3/11.3 MB 11.6 MB/s eta 0:00:01
   ---------------------------------- ----- 9.7/11.3 MB 11.8 MB/s eta 0:00:01
   ---------------------------------------- 11.3/11.3 MB 11.4 MB/s  0:00:01

   ---------------------------------------- 0/3 [pytz]
   ------------- -------------------------- 1/3 [tzdata]
   ------------- -------------------------- 1/3 [tzdata]
   -------------------------- ------------- 2/3 [pandas]
   -------------------------- ------------- 2/3 [pandas]
   -------------------------- ------------- 2/3 [pandas]
   -------------------------- ------------- 2/3 [pandas]
   -------------------------- ------------- 2/3 [pandas]
   -------------------------- ------------- 2/3 [pandas]
   -------------------------- 

In [3]:
# bo_lognei_latest.py
import os, sys, csv
from datetime import datetime
from pathlib import Path
from typing import Dict, Tuple, Optional

import torch
from torch import Tensor

# ---- BoTorch / GPyTorch ----
from botorch.models import SingleTaskGP
from botorch.models.transforms import Standardize, Normalize
from botorch.fit import fit_gpytorch_mll
from gpytorch.mlls import ExactMarginalLogLikelihood

from botorch.acquisition.monte_carlo import qUpperConfidenceBound
from botorch.acquisition.logei import qLogNoisyExpectedImprovement
from botorch.sampling.normal import SobolQMCNormalSampler
from botorch.optim import optimize_acqf

In [4]:
# =========================
# User Config (EDIT HERE)
# =========================
SEED = 123
MAX_ITERS = 20                 # 총 이터레이션 수
N_RANDOM = 5                   # 초기 랜덤 탐색 이터레이션
N_UCB = 5                      # 그 다음 UCB 이터레이션
UCB_BETA = 0.25                # qUCB beta
OBJECTIVE_SENSE = "min"        # "min" or "max"
LOG_CSV = "bo_log.csv"         # 로그 파일 경로

# 파라미터 범위 (예시) → 네 범위로 수정
PBONDS: Dict[str, Tuple[float, float]] = {
    "pressure"     : (250.0, 450.0),
    "velocity"     : (2.0, 40.0),
    "wall_spacing" : (0.20, 1.00),
    "layer_spacing": (0.10, 0.60),
}

In [5]:
# =========================
# Helpers
# =========================
torch.manual_seed(SEED)
tkwargs = {"dtype": torch.double, "device": "cpu"}

PARAMS = list(PBONDS.keys())
LB = torch.tensor([PBONDS[p][0] for p in PARAMS], **tkwargs)
UB = torch.tensor([PBONDS[p][1] for p in PARAMS], **tkwargs)
BOUNDS = torch.stack([LB, UB])  # shape [2, d]
D = len(PARAMS)


In [6]:
def ensure_csv(path: str):
    """Create CSV with header if missing."""
    if not Path(path).exists():
        with open(path, "w", newline="", encoding="utf-8") as f:
            w = csv.writer(f)
            header = ["iter", "timestamp"] + PARAMS + [
                "objective_raw", "objective_for_BO", "acquisition"
            ]
            w.writerow(header)


def log_row(path: str, iteration: int, x: Tensor, y_raw: float, y_bo: float, acq_name: str):
    """Append one row to CSV."""
    ts = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    x_list = [float(v) for v in x.view(-1).tolist()]
    row = [iteration, ts] + x_list + [float(y_raw), float(y_bo), acq_name]
    with open(path, "a", newline="", encoding="utf-8") as f:
        csv.writer(f).writerow(row)


def sample_random(n: int = 1) -> Tensor:
    """Uniform random sample within bounds."""
    u = torch.rand(n, D, **tkwargs)
    return LB + (UB - LB) * u


def ask_user_evaluate(x: Tensor) -> float:
    """Print x and ask user for objective value."""
    print("\n--- Evaluate Candidate ---")
    for i, p in enumerate(PARAMS):
        print(f"{p:>14s} : {float(x[i]):.6g}")
    while True:
        s = input("Enter objective value (float): ").strip()
        try:
            return float(s)
        except ValueError:
            print("⚠️  Not a float. Try again.")


def y_for_bo(y_raw: float) -> float:
    """Convert raw objective to 'maximize' convention for BoTorch."""
    if OBJECTIVE_SENSE.lower().startswith("min"):
        return -float(y_raw)  # minimize → maximize by sign flip
    return float(y_raw)


def fit_model(train_X: Tensor, train_Y: Tensor) -> SingleTaskGP:
    """Fit SingleTaskGP with normalization/standardization."""
    model = SingleTaskGP(
        train_X,
        train_Y,
        input_transform=Normalize(d=D),
        outcome_transform=Standardize(m=1),
    )
    mll = ExactMarginalLogLikelihood(model.likelihood, model)
    fit_gpytorch_mll(mll)
    return model


def next_via_ucb(model: SingleTaskGP, q: int = 1) -> Tensor:
    sampler = SobolQMCNormalSampler(sample_shape=torch.Size([512]))
    acq = qUpperConfidenceBound(model, beta=UCB_BETA, sampler=sampler)
    cand, _ = optimize_acqf(
        acq_function=acq,
        bounds=BOUNDS,
        q=q,
        num_restarts=10,
        raw_samples=256,
        options={"batch_limit": 5, "maxiter": 200},
    )
    return cand.detach()


def next_via_lognei(model: SingleTaskGP, train_X: Tensor, train_Y: Tensor, q: int = 1) -> Tensor:
    sampler = SobolQMCNormalSampler(sample_shape=torch.Size([1024]))
    acq = qLogNoisyExpectedImprovement(
        model=model,
        X_baseline=train_X,
        sampler=sampler,
    )
    cand, _ = optimize_acqf(
        acq_function=acq,
        bounds=BOUNDS,
        q=q,
        num_restarts=15,
        raw_samples=256,
        options={"batch_limit": 5, "maxiter": 200},
    )
    return cand.detach()


def tensorize_logs(path: str) -> Tuple[Optional[Tensor], Optional[Tensor]]:
    """Load existing CSV (if any) to X,Y tensors for warm start."""
    if not Path(path).exists():
        return None, None
    import pandas as pd
    df = pd.read_csv(path)
    if df.empty:
        return None, None
    X = torch.tensor(df[PARAMS].values, **tkwargs)
    Y_bo = torch.tensor(df["objective_for_BO"].values.reshape(-1, 1), **tkwargs)
    return X, Y_bo


def main():
    ensure_csv(LOG_CSV)

    # Warm-start from existing log (optional)
    X_all, Y_all = tensorize_logs(LOG_CSV)
    if X_all is None:
        X_all = torch.empty(0, D, **tkwargs)
        Y_all = torch.empty(0, 1, **tkwargs)
        iter_start = 1
    else:
        iter_start = int(X_all.shape[0]) + 1
        print(f"✅ Resuming from {LOG_CSV}: {iter_start-1} rows loaded.")

    # BO Loop
    for it in range(iter_start, MAX_ITERS + 1):
        if it <= N_RANDOM:
            acq_name = "RANDOM"
            x_next = sample_random(1).squeeze(0)
        else:
            # need at least 2 points to fit GP sensibly
            if X_all.shape[0] < 2:
                x_next = sample_random(1).squeeze(0)
                acq_name = "RANDOM"
            else:
                # Fit model on current data
                try:
                    model = fit_model(X_all, Y_all)
                except Exception as e:
                    print(f"⚠️ GP fit failed ({e}). Fallback to RANDOM.")
                    x_next = sample_random(1).squeeze(0)
                    acq_name = "RANDOM"
                else:
                    if it <= N_RANDOM + N_UCB:
                        acq_name = f"qUCB(beta={UCB_BETA})"
                        x_next = next_via_ucb(model).squeeze(0)
                    else:
                        acq_name = "qLogNEI"
                        x_next = next_via_lognei(model, X_all, Y_all).squeeze(0)

        # Evaluate (user input)
        y_raw = ask_user_evaluate(x_next)
        y_bo = y_for_bo(y_raw)

        # Update data
        X_all = torch.cat([X_all, x_next.view(1, -1)], dim=0)
        Y_all = torch.cat([Y_all, torch.tensor([[y_bo]], **tkwargs)], dim=0)

        # Log to CSV
        log_row(LOG_CSV, it, x_next, y_raw, y_bo, acq_name)
        print(f"📎 Logged iter {it} ({acq_name}) → raw={y_raw:.6g}  for_BO={y_bo:.6g}")

    print("\n🎉 BO finished.")
    print(f"Log saved to: {Path(LOG_CSV).resolve()}")



In [ ]:
try:
    main()
except KeyboardInterrupt:
    print("\nInterrupted by user.")
    sys.exit(0)

✅ Resuming from bo_log.csv: 5 rows loaded.

--- Evaluate Candidate ---
      pressure : 329.859
      velocity : 2
  wall_spacing : 0.579741
 layer_spacing : 0.221724
Enter objective value (float): 15
📎 Logged iter 6 (qUCB(beta=0.25)) → raw=15  for_BO=-15

--- Evaluate Candidate ---
      pressure : 316.871
      velocity : 2
  wall_spacing : 0.68267
 layer_spacing : 0.136228
Enter objective value (float): 13
📎 Logged iter 7 (qUCB(beta=0.25)) → raw=13  for_BO=-13

--- Evaluate Candidate ---
      pressure : 319.246
      velocity : 2.62905
  wall_spacing : 0.655137
 layer_spacing : 0.151644
Enter objective value (float): 12
📎 Logged iter 8 (qUCB(beta=0.25)) → raw=12  for_BO=-12

--- Evaluate Candidate ---
      pressure : 368.86
      velocity : 3.33691
  wall_spacing : 0.727375
 layer_spacing : 0.145501
Enter objective value (float): 13
📎 Logged iter 9 (qUCB(beta=0.25)) → raw=13  for_BO=-13

--- Evaluate Candidate ---
      pressure : 450
      velocity : 4.44424
  wall_spacing : 0.91